# Bootstrapping

Runs the process of measuring every single quantity used in other notebooks through a hundred bootstrap iterations, restricting sample of 90% each time to generate uncertainty estimates from the sample.

In [1]:
import illustris_python as il
import numpy as np
import h5py
from os.path import exists 
from numpy.linalg import inv
import seaborn as sns
import pandas as pd
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm
from scipy.interpolate import CubicSpline

import sys
sys.path.append('/home/tnguser/python/')
import numpy as np
import h5py
import os
from FigureRotation_util import *
from Cannon_get_principal_axes import *
from prob_plane_method import *

from astropy import constants as const
from astropy import units as u
from matplotlib import rcParams
import warnings
warnings.filterwarnings("ignore")
import agama

%matplotlib inline
rcParams['font.size'] = 18

sim = 'L35n2160TNG'
basePath = '/home/tnguser/sims.TNG/' + sim + '/output'

# Halos in Neil Ash's catalog
catalogue_path = '/home/tnguser/postprocessing/halocatalogues/' + sim + '.npy'
naive_halos = np.load(catalogue_path)
main_subhalos = np.load('/home/tnguser/postprocessing/halocatalogues/' + sim + '_mainSubhalos.npy') 

# CHANGE TO DESIRED FILE PATHS
anglePath = '/home/tnguser/postprocessing/angles/' + sim + '/00_15Rvir/'
spin_path = '/home/tnguser/postprocessing/angular_momentum/' + sim + '/inner_disk/'
pattern_path =  '/home/tnguser/postprocessing/pattern_speeds/' + sim + '/00_06Rvir/'
principal_path =  '/home/tnguser/postprocessing/principal_axes/' + sim + '/00_06Rvir/'
spin_suffix = '/home/tnguser/postprocessing/angular_momentum/' + sim + '/rings_log/'
mag_path = '/home/tnguser/postprocessing/warp_magnitudes/'

snap = 99
startSnap = 75

In [2]:
def getAngle(vec1, vec2):
    return np.degrees(np.arccos(np.dot(vec1, vec2)))

def getAngles(v):
    """
    Returns the Brigg's angles in form [polar angle, azmuthal angle]
    """
    polar_angle = angleBtwn(v,[0,0,1])
    xy_angle = np.sign(v[1])*np.arccos(v[0]/np.sqrt(v[0]**2+v[1]**2))
    return [polar_angle, xy_angle]

def angleBtwn(u,v):
    """
    Returns angle between vectors u and v
    """
    u = np.array(u)
    v= np.array(v)
    u = 1/np.sqrt(np.sum(u**2))*u
    v = 1/np.sqrt(np.sum(v**2))*v
    
    return np.arccos(np.dot(u,v))

def getRotationAxis(snap, startStops, Raxis):
    for idx, (first,second) in enumerate(startStops):
        if (snap >= first and snap < second):
            return Raxis[idx]
    return Raxis[-1]

def Pearson_correlation(X,Y):
    if len(X)==len(Y):
        Sum_xy = sum((X-X.mean())*(Y-Y.mean()))
        Sum_x_squared = sum((X-X.mean())**2)
        Sum_y_squared = sum((Y-Y.mean())**2)       
        corr = Sum_xy / np.sqrt(Sum_x_squared * Sum_y_squared)
    return corr

def getIndex(list, num):
    for idx, val in enumerate(list):
        if(num in range(val[0],val[1])):
            return idx
    return -1

def snapshot_redshift_corr(basePath,startSnap=75):
    """
    Calls and stores z = and redshift values
    """
    redshift_space = []
    for snapshot_number in range(startSnap,100):
        header=il.groupcat.loadHeader(basePath,snapshot_number) 
        redshift_space.append(header.get('Redshift')) 
    return np.arange(startSnap,100), np.array(redshift_space) 

# Helper funcation which calculates Briggs angles for each angular momentum axis in array spins

def getBriggsAngles(spins, GrNr, subfindID):
    
    principal_file = principal_path + 'GrNr_%d_snap_%d_99_principal_axes_full.npy'%(GrNr,75)
    pattern_file = pattern_path + 'GrNr_%d_snap_%d_99_patternSpeeds.npy'%(GrNr,75)
    
    theta = []; phi = []
    if exists(principal_file) and exists(pattern_file): #crucially needs both files on the same index to calculate Briggs
        
        principal_axes = np.load(principal_file,allow_pickle=True)
        pattern_info = np.load(pattern_file, allow_pickle=True)
        pattern_info = dict(enumerate(pattern_info.flatten(),1))[1]
        
        raxis = getRotationAxis(25, pattern_info['startStop'], pattern_info['Raxis'])
        raxis = 1/np.sqrt(np.sum(raxis**2))*raxis
        
        # Finds transformation matrix that rotates figure rotation axis to align with z-axis, tries to align
        # halo major axis with x-axis
        z = [[0, 0, 1], [1, 0, 0]]
        rot, rssd, sens = R.align_vectors(z, [raxis, [1,0,0]], return_sensitivity=True, weights=[100, .1])
        for snap in range(25):
            for spin in spins:
                spin_bf = np.dot(inv(principal_axes[snap]), spin) # Changes to body frame coordinates
                spin_new = (rot.apply(spin_bf)) # Then applies new rotation 
                spin_new = 1/np.sqrt(np.sum(spin_new**2))*spin_new

                # Theta (polar angle): angle between fiducial z-axis (figure rotation axis)
                # Phi (azimuthal angle): angle between projection onto xy-plane and x-axis (near halo major axis)
                t,p = getAngles(spin_new)
                theta.append(t)
                phi.append(p)

    return theta, phi

def make_unit(vec):
    """
    Convenience function, returns the unit-normalized vector
    """
    return 1/np.sqrt(np.sum(vec**2)) * vec

def get_indices(lst, targets):
    return list(filter(lambda x: lst[x] in targets, range(len(lst))))

def func(x,m,b):
    return m*x + b

In [3]:
class decomposeDisk:
    
    def __init__(self,coords,mass,bins,mmax=4):
        """
        Decompose the vertical structure of the disk into an Azimuthal-Harmonic basis set.
        Inputs:
                coords: Cartesian (x,y,z) positions of the disk particles, assumed to be 
                        centered and rotated such that the central angular momentum axis 
                        lies along z
                        
                mass:   Masses of each particle
                
                bins:   annular bins in which to perform the expansion
                
                mmax:   maximum m order of the expansion. Usually higher than 1 is not needed, 
                        and may be excessively noisy currently
                        
        Outputs: 
                None
        
        """
        # Convert to cylindrical
        self.R = np.sqrt(coords[:,0]**2+coords[:,1]**2)
        self.Theta = np.arctan2(coords[:,1],coords[:,0])
        self.Z = coords[:,2]
        self.mass = mass
        self.mmax = mmax
        
        # Set up loop through bins, run decomposition on each
        self.z_m_R = []
        self.bin_centers = []
        for i in range(len(bins)-1): 
            bin_indices = (self.R>bins[i])&(self.R<bins[i+1])
            self.z_m_R.append(self.decompose(bin_indices))
            self.bin_centers.append(np.sum(mass[bin_indices]*self.R[bin_indices])/np.sum(mass[bin_indices]))
        
        # Get the mag and phase angle of coefficients
        self.z_m_R = np.array(self.z_m_R)
        self.z_m_R_mag = np.real(np.sqrt(self.z_m_R*np.conj(self.z_m_R)))
        self.z_m_R_PA = 1/np.arange(mmax+1)*np.arctan2(np.imag(self.z_m_R),np.real(self.z_m_R))
        
        # mean particle position in each bin,
        self.bin_centers = np.array(self.bin_centers)
        self.z_m_Spl = CubicSpline(self.bin_centers,self.z_m_R)
        
    def decompose(self,indices):
        """
        Performs the decomposition into the basis set coefficient weights, 
        should only be used internally by __init__
        
        Inputs:
                indices: the indices into coords and mass to be considered in the decomposition
                
        Outputs:
                z_m_arr: the coefficient weights
        
        """
        z_m_arr = []
        for m in range(0,self.mmax+1,1):
            z_m = np.sum(self.mass[indices]*self.Z[indices]*np.exp(1j*m*self.Theta[indices])) /\
                    np.sum(self.mass[indices])
            if m != 0: z_m *= 2
            z_m_arr.append(z_m)
        return np.array(z_m_arr)        
        
    def reconstruct_disk(self,pos,m_inds=None):
        """
        Recover the model z-height above/below the nominal disk plane for selected m-modes
        Inputs: 
                pos:      array-like grid of points to return z-height along
                m_inds:   array-like or int, m-modes to use in model reconstruction
            
        Outputs:
                z_recov:  array of same shape as pos, the model z-value at pos
        """
        m_arr = np.arange(self.mmax+1)
        
        # Convert to cylindrical 
        R = np.sqrt(pos[:,0]**2+pos[:,1]**2)
        Theta = np.arctan2(pos[:,1],pos[:,0])
        
        # Get the interpolated coefficients
        z_m = self.z_m_Spl(R)
        
        # Recover phase angle and magnitude of coefficients
        z_phi = 1/m_arr * np.arctan2(np.imag(z_m),np.real(z_m))
        z_mag = np.real(np.sqrt(z_m*np.conj(z_m)))
        
        # Reconstruct z-profile, sum contribution from each m
        if m_inds is None:
            m_inds = m_arr
            
        z_recov_m = np.array([z_mag[:,m]*np.cos(m_arr[m]*(Theta-z_phi[:,m])) for m in m_arr])
        return np.nansum(z_recov_m[m_inds,:],axis=0)

In [4]:
def orient(E,E_ref=np.eye(3),assert_on=True):
    """
    Enforces E to be a RHS, then attempts rotations by 180 deg about each axis until
    the angles with respect to E_ref are minimized. If any of these cannot be reduced beyond 90 deg,
    throw error
    """

    # Enforce RHS
    RHS = np.dot(np.cross(E[:,0],E[:,1]),E[:,2]) > 0
    if not RHS: E[:,0] = E[:,0]*-1

    # Begin iteration
    # Check base orientation first
    rotations = [[ 1, 1, 1],
                 [-1,-1, 1],
                 [ 1,-1,-1],
                 [-1, 1,-1]]

    angles = []
    for rot in rotations:
        E_rot = E * rot

        # Check angles between E, E_ref
        angles.append(np.arccos(np.diagonal(E_rot.T.dot(E_ref))))

    # Find the rotation which gives us the smallest square error from E_ref
    angles = np.array(angles)
    mean_sqr_dev = np.mean(angles**2,axis=1)

    min_ind = np.argmin(mean_sqr_dev)
    E_new = E*rotations[min_ind]

    # Verify none of the angles on this surpass 90 deg
    #print(angles[min_ind]*180/np.pi)
    if assert_on:
        assert np.all(angles[min_ind]<np.pi/2)

    return E_new

def RotateForPolarPlot(disk_spin_axes,halo_principal_axes,
                       close_to_unit_thresh=1e-4,useMajorAxis=False):
    """
    A function to take the orientations of the disk spin axes and 
    halo minor axes and perform the rotation for plotting on polar 
    plots. The same rotation is applied to both sets of axes, such 
    that they are kept in the same frame. 
    
    Additionally, the orientation of the disk spin axis in the halo 
    body frame is returned
    
    Inputs:
            disk_spin_axes: (N,3), representing the orientation of 
                                    the inner disk
            
            halo_principal_axes: (N,3,3), the principal axes of the 
                                            halo at all snapshots
            
    Outputs:
            
            disk_theta_phi: (N,2), angular coordinates of the disk 
                                    spin axis, to be used for plotting
            
            halo_theta_phi: (N,2), angular coordinates of the halo minor 
                                    axis, to be used for plotting
            
            disk_theta_phi_in_halo_body_frame: (N,2), angular coordinates 
                                    of the disk spin axis in the halo
    
    """
    ##################################################
    #### Preprocessing block                      ####
    ##################################################
    
    # First, check input shapes
    N = len(disk_spin_axes)
    
    assert N==len(halo_principal_axes), "Length of inputs must be equal"
    assert disk_spin_axes.shape==(N,3), "Disk spin axes must have shape (N,3), but found "+str(disk_spin_axes.shape)
    assert halo_principal_axes.shape==(N,3,3), "Halo principal axes must have shape (N,3,3), but found "+str(halo_principal_axes.shape)
    
    # Second, do some checks on the axes
    
    # Are all spin axes unit vectors?
    spin_mag = np.sqrt(np.sum(disk_spin_axes**2,axis=1))
    assert all(abs(spin_mag-1)<close_to_unit_thresh), "Disk spin axes are not unit length"
    
    # Are the principal axes all RHS and unit vectors?
    halo_PA_det = np.linalg.det(halo_principal_axes)
    assert all(halo_PA_det>0), "Principal axes must make up a RHS"
    
    # Are all the principal axes unit length?
    assert all(abs(halo_PA_det-1)<close_to_unit_thresh), "Principal axes must be mutually orthogonal unit vectors"
    
    # Check that all the principal axes have been oriented correctly
    halo_PA_orient = [halo_principal_axes[0]]
    for j in range(1,N):
        halo_PA_orient.append(orient(halo_principal_axes[j],E_ref=halo_PA_orient[j-1]))
    halo_PA_orient = np.array(halo_PA_orient)
    
    # Check that the halo minor axis is not opposite the disk spin axis
    # If it is, flip minor and intermediate axes
    if useMajorAxis:
        halo_minor_axes = halo_PA_orient[:,:,2]
    else:
        halo_minor_axes = halo_PA_orient[:,:,2]
    
    dot_prod = np.sum(halo_minor_axes*disk_spin_axes,axis=1)
    if np.mean(dot_prod)<0:
        halo_PA_orient *= np.array([1,-1,-1]).reshape((1,1,3))
        
    ##################################################
    #### Frame def block                          ####
    ##################################################
    
    # Define the rotation matrix
    # First, compute the covariance matrix of the stellar spin axes
    covMat = disk_spin_axes.T.dot(disk_spin_axes)
    
    # eigen decompose the covariance matrix
    val,vec = np.linalg.eig(covMat)
    
    # Rearrange by variance, so that the z axis is the first eigenvector,
    # x axis the second eigenvector, and y axis the third
    sortInds = np.argsort(val)
    #sortInds[1:] = sortInds[1:][::-1]
    
    vec = vec[:,[sortInds[1],sortInds[0],sortInds[2]]]
    
    # Check that the first eigenvector is mostly aligned with 
    # the stellar spin axes
    if np.mean(np.sum(vec[:,2]*disk_spin_axes,axis=1))<0:
        vec[:,2] *= -1
        
    # Check that the eigenvectors form a RHS, correct if not
    if np.linalg.det(vec)<0:
        vec = vec*np.array([-1,-1,1])
        
    # Finally, define the rotation matrix
    R = vec.T
    
    ##################################################
    #### Coordinate transform block               ####
    ##################################################
    
    # First the disk spin axes
    disk_rot = R.dot(disk_spin_axes.T).T
    
    theta = np.arccos(disk_rot[:,2])
    phi = np.arctan2(disk_rot[:,1],disk_rot[:,0])
    
    disk_theta_phi = np.array([theta,phi])
    
    # Next halo minor axes
    if useMajorAxis:
        halo_minor_axes = halo_PA_orient[:,:,0]
    else:
        halo_minor_axes = halo_PA_orient[:,:,2]
    halo_rot = R.dot(halo_minor_axes.T).T
    
    theta = np.arccos(halo_rot[:,2])
    phi = np.arctan2(halo_rot[:,1],halo_rot[:,0])
    
    halo_theta_phi = np.array([theta,phi])
    
    # Finally, output the disk spin axis orientation in 
    # the halo body frame
    
    disk_in_body_frame = np.array([halo_PA_orient[i].T.dot(disk_spin_axes[i]) for i in range(N)])
    
    theta = np.arccos(disk_in_body_frame[:,2])
    phi = np.arctan2(disk_in_body_frame[:,1],disk_in_body_frame[:,0])
    
    disk_theta_phi_in_halo_body_frame = np.array([theta,phi])
    
    # output
    
    return disk_theta_phi, halo_theta_phi, disk_theta_phi_in_halo_body_frame

In [6]:
final_diskyIDs = np.load('/home/tnguser/postprocessing/circularity_study/final_diskyIDs.npy')

In [7]:
p = .9
N = 100
#Let's calculate these according to half mass radius instead

save_name = '/home/tnguser/postprocessing/uncertainties/angles/'

for subfindID in tqdm(final_diskyIDs[48:-4]):

    ### CODE IN COMMENTS ADAPTED FROM NEIL'S "measure_halo_spin.py", 75-99 non-tilted 3kpc ----------------------------------------------
    ### ----------------------------------------------------------------------------------------------------------
    GrNr = (il.groupcat.loadSingle(basePath, 99, subhaloID=subfindID)['SubhaloGrNr'])

    snapArr,zArr = snapshot_redshift_corr(basePath)

    haloTree = il.lhalotree.loadTree(basePath,99,subfindID,
                         fields=['SubhaloGrNr','SnapNum','Group_R_Crit200','SubhaloPos','SubhaloNumber','SubhaloHalfmassRadType'], 
                         onlyMPB=True)

    haloInd,mpb_snapArr = haloTree['SubhaloGrNr'], haloTree['SnapNum']
    subfindID = haloTree['SubhaloNumber']
    Rvirial   = haloTree['Group_R_Crit200']
    subhaloPos = haloTree['SubhaloPos']
    halfmassrad = haloTree['SubhaloHalfmassRadType']
    angles_mean = []
    angles_std = []
    for snap in (range(75,100)):

        GrNr_i       = value(haloInd[snap == mpb_snapArr])
        subfindID_i  = value(subfindID[snap == mpb_snapArr])
        Rvirial_i    = value(Rvirial[snap == mpb_snapArr])
        subhaloPos_i = value(subhaloPos[snap == mpb_snapArr])
        a = 1/(1+value(zArr[snap==snapArr]))
        halfmassrad_i = value(halfmassrad[snap == mpb_snapArr])[4]*a

        starPos = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Coordinates')
        starVel = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Velocities')
        starMass = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Masses') 

        gasPos = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,0,fields='Coordinates')
        gasVel = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,0,fields='Velocities')
        gasMass = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,0,fields='Masses')    
        
        particleMass_bar    = starMass * 1e10
        particleMass_gas    = gasMass * 1e10
        particleCoords_bar = starPos
        particleCoords_gas = gasPos
        particleVels_bar = starVel
        particleVels_gas = gasVel

        # center coords on subhalo pos
        particleCoords_bar = (particleCoords_bar - subhaloPos_i)*a
        # find relative velocities - shouldn't have done this, instead use the spin by TNG (includes DM)
        particleVels_bar = (particleVels_bar - np.mean(particleVels_bar,axis=0)) * np.sqrt(a) # km/s

        # center coords on subhalo pos
        particleCoords_gas = (particleCoords_gas - subhaloPos_i)*a
        # find relative velocities - shouldn't have done this, instead use the spin by TNG (includes DM)
        particleVels_gas = (particleVels_gas - np.mean(particleVels_gas,axis=0)) * np.sqrt(a) # km/s
        
        angles = []
        innerRad = 0
        outerRad = halfmassrad_i
        for i in (np.arange(N)):
            r_bar = np.sqrt(np.sum(particleCoords_bar**2,axis=1))
            indsInBounds = np.where((r_bar>innerRad)&(r_bar<=outerRad))[0]        

            # Choosing p percent of samples with repetition
            num_data = len(indsInBounds)
            num_samples = int(p * num_data)
            random_indices = (np.random.randint(0,num_data,size=num_samples))
            bootstrap_inds = np.array(indsInBounds)[random_indices.astype(int)]

            J_bar = np.sum(particleMass_bar[bootstrap_inds].reshape(-1,1)*np.cross(particleCoords_bar[bootstrap_inds],
                                                                                    particleVels_bar[bootstrap_inds]),axis=0) 
            L_spin = make_unit(J_bar)

            r_gas = np.sqrt(np.sum(particleCoords_gas**2,axis=1))
            indsInBounds = np.where((r_gas>0)&(r_gas<=10*halfmassrad_i))[0]        

            # Choosing p percent of samples with repetition
            num_data = len(indsInBounds)
            num_samples = int(p * num_data)
            random_indices = (np.random.randint(0,num_data,size=num_samples))
            bootstrap_inds = np.array(indsInBounds)[random_indices.astype(int)]

            J_gas = np.sum(particleMass_gas[bootstrap_inds].reshape(-1,1)*np.cross(particleCoords_gas[bootstrap_inds],
                                                                                    particleVels_gas[bootstrap_inds]),axis=0) 
            gas_spin = make_unit(J_gas)

            angle = np.degrees(np.arccos(np.abs(np.dot(gas_spin, L_spin))))
            angles.append(angle)

        angles_mean.append(np.mean(angles))
        angles_std.append(np.std(angles))

    temp = save_name+'GrNr_%d_snap_%d_99_LGaxisL*axis_angle_std.npy'%(GrNr,75)
    np.save(temp,angles_std,allow_pickle=True)

100%|██████████| 34/34 [26:21<00:00, 46.51s/it]


In [8]:
p = .9
N = 100
save_name = '/home/tnguser/postprocessing/uncertainties/warp_angles/'
for subfindID in tqdm(np.sort(final_diskyIDs)[80:]):

    GrNr = (il.groupcat.loadSingle(basePath, 99, subhaloID=subfindID)['SubhaloGrNr'])

    snapArr,zArr = snapshot_redshift_corr(basePath)

    haloTree = il.lhalotree.loadTree(basePath,99,subfindID,
                                     fields=['SubhaloGrNr','SnapNum','Group_R_Crit200','SubhaloPos','SubhaloNumber','SubhaloHalfmassRadType'], 
                                     onlyMPB=True)

    haloInd,mpb_snapArr = haloTree['SubhaloGrNr'], haloTree['SnapNum']
    subfindID = haloTree['SubhaloNumber']
    Rvirial   = haloTree['Group_R_Crit200']
    subhaloPos = haloTree['SubhaloPos']
    halfmassrad = haloTree['SubhaloHalfmassRadType']
    angles_mean = []
    angles_std = []

    for snap in (range(75,100)):
        GrNr_i       = value(haloInd[snap == mpb_snapArr])
        subfindID_i  = value(subfindID[snap == mpb_snapArr])
        Rvirial_i    = value(Rvirial[snap == mpb_snapArr])
        subhaloPos_i = value(subhaloPos[snap == mpb_snapArr])
        a = 1/(1+value(zArr[snap==snapArr]))
        halfmassrad_i = value(halfmassrad[snap == mpb_snapArr])[4]*a

        starPos = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Coordinates')
        gasPos = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,0,fields='Coordinates')
        dmPos = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,1,fields='Coordinates')

        starVel = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Velocities')
        gasVel = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,0,fields='Velocities')
        dmVel = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,1,fields='Velocities')

        starMass = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Masses') 

            # following Neil's method, but only using stars (not gas or DM)
        if (type(starPos) is not dict):
            particleCoords_bar  = starPos
            particleVels_bar    = starVel
            particleMass_bar    = starMass * 1e10
        else:
                # no stars in the halo
            print("ERROR: no stars in halo.")
            particleCoords_bar  = np.array([[np.nan,np.nan,np.nan]])
            particleVels_bar    = np.array([[np.nan,np.nan,np.nan]])
            particleMass_bar    = np.array([np.nan])

        if (type(gasPos) is not dict):
            particleCoords_gas  = gasPos
            particleVels_gas    = gasVel

        else:
                # no stars in the halo
            print("ERROR: no gas in halo.")
            particleCoords_gas  = np.array([[np.nan,np.nan,np.nan]])
            particleVels_gas    = np.array([[np.nan,np.nan,np.nan]])

        if (type(gasPos) is not dict):
            particleCoords_dm  = dmPos
            particleVels_dm   = dmVel

        else:
                # no stars in the halo
            print("ERROR: no dark matter in halo.")
            particleCoords_dm  = np.array([[np.nan,np.nan,np.nan]])
            particleVels_dm    = np.array([[np.nan,np.nan,np.nan]])

            ########################################################################
            # check that the halo is not crossing the box boundary, correct if it is
            # condition is whether it is within 2 virial radii
        passUpperBound = subhaloPos_i + 2*Rvirial_i >= 35000
        passLowerBound = subhaloPos_i - 2*Rvirial_i <= 0

        if any(passUpperBound):
                # shift the particles
            particleCoords_bar[:,passUpperBound] = particleCoords_bar[:,passUpperBound] - 35000/2
            subhaloPos_i[passUpperBound] = subhaloPos_i[passUpperBound] - 35000/2
                #reinforce boundary condition
            ind_bar = np.where(particleCoords_bar[:,passUpperBound]<0)[0]
            particleCoords_bar[ind_bar,passUpperBound] = particleCoords_bar[ind_bar,passUpperBound] + 35000

        if any(passLowerBound):
            particleCoords_bar[:,passLowerBound] = particleCoords_bar[:,passLowerBound] + 35000/2 
            subhaloPos_i[passLowerBound] = subhaloPos_i[passLowerBound] + 35000/2

            ind_bar = np.where(particleCoords_bar[:,passLowerBound]>35000)[0]
            particleCoords_bar[ind_bar,passLowerBound] = particleCoords_bar[ind_bar,passLowerBound] - 35000
            ########################################################################

            # center coords on subhalo pos
        particleCoords_bar = (particleCoords_bar - subhaloPos_i)*a
            # find relative velocities - shouldn't have done this, instead use the spin by TNG (includes DM)
        paricleVels_bar = (particleVels_bar - np.mean(particleVels_bar,axis=0)) * np.sqrt(a) # km/s

        if any(passUpperBound):
                # shift the particles
            particleCoords_gas[:,passUpperBound] = particleCoords_gas[:,passUpperBound] - 35000/2
            subhaloPos_i[passUpperBound] = subhaloPos_i[passUpperBound] - 35000/2
                #reinforce boundary condition
            ind_bar = np.where(particleCoords_gas[:,passUpperBound]<0)[0]
            particleCoords_gas[ind_bar,passUpperBound] = particleCoords_gas[ind_bar,passUpperBound] + 35000

        if any(passLowerBound):
            particleCoords_gas[:,passLowerBound] = particleCoords_gas[:,passLowerBound] + 35000/2 
            subhaloPos_i[passLowerBound] = subhaloPos_i[passLowerBound] + 35000/2

            ind_bar = np.where(particleCoords_gas[:,passLowerBound]>35000)[0]
            particleCoords_gas[ind_bar,passLowerBound] = particleCoords_gas[ind_bar,passLowerBound] - 35000
            ########################################################################

            # center coords on subhalo pos
        particleCoords_gas = (particleCoords_gas - subhaloPos_i)*a
        particleVels_gas = (particleVels_gas - np.mean(particleVels_gas,axis=0)) * np.sqrt(a) # km/s

        if any(passUpperBound):
                # shift the particles
            particleCoords_dm[:,passUpperBound] = particleCoords_dm[:,passUpperBound] - 35000/2
            subhaloPos_i[passUpperBound] = subhaloPos_i[passUpperBound] - 35000/2
                #reinforce boundary condition
            ind_bar = np.where(particleCoords_dm[:,passUpperBound]<0)[0]
            particleCoords_dm[ind_bar,passUpperBound] = particleCoords_dm[ind_bar,passUpperBound] + 35000

        if any(passLowerBound):
            particleCoords_dm[:,passLowerBound] = particleCoords_dm[:,passLowerBound] + 35000/2 
            subhaloPos_i[passLowerBound] = subhaloPos_i[passLowerBound] + 35000/2

            ind_bar = np.where(particleCoords_dm[:,passLowerBound]>35000)[0]
            particleCoords_dm[ind_bar,passLowerBound] = particleCoords_dm[ind_bar,passLowerBound] - 35000
            ########################################################################

            # center coords on subhalo pos
        particleCoords_dm = (particleCoords_dm - subhaloPos_i)*a
        particleVels_dm = (particleVels_dm - np.mean(particleVels_dm,axis=0)) * np.sqrt(a) # km/s

        # Rotate into plane of inner disk
        r = np.sqrt(np.sum(particleCoords_bar**2,axis=1))
        in_half = r<=halfmassrad_i

        L_parts = particleMass_bar.reshape(-1,1) * np.cross(particleCoords_bar,
                                                              paricleVels_bar)
        L_spec_half = (np.sum(L_parts[in_half],axis=0) / np.sum(particleMass_bar[in_half]).reshape(-1,1)).reshape(3)

        e3 = L_spec_half / np.sqrt(np.sum(L_spec_half**2))
        e1 = np.cross(np.array([0,1,0]),e3)
        e1 = e1 / np.sqrt(np.sum(e1**2))
        e2 = np.cross(e3,e1)

        Rmat = np.array([e1,e2,e3]).T

        # Rotate
        particleCoords_bar = Rmat.T.dot(particleCoords_bar.T).T
        particleVels_bar = Rmat.T.dot(particleVels_bar.T).T

        particleCoords_gas = Rmat.T.dot(particleCoords_gas.T).T
        particleVels_gas = Rmat.T.dot(particleVels_gas.T).T

        particleCoords_dm = Rmat.T.dot(particleCoords_dm.T).T
        particleVels_dm = Rmat.T.dot(particleVels_dm.T).T
        # -------------------------------------------------------------------------------------------------------

        # BOOTSTRAPPING HERE
        logBins = np.logspace(np.log10(halfmassrad_i),np.log10(2*halfmassrad_i),10)
        mmax = 3
        all_angles = []
        for i in np.arange(N):
            # Choosing p percent of samples with repetition
            num_data = len(particleCoords_bar)
            num_samples = int(p * num_data)
            random_indices = (np.random.randint(0,num_data,size=num_samples))
            warpModel = decomposeDisk(particleCoords_bar[random_indices],particleMass_bar[random_indices],logBins,mmax=mmax)
            popt, pcov = curve_fit(func, warpModel.bin_centers, warpModel.z_m_R_mag[:,1])

            all_angles.append(np.abs(np.degrees(np.arctan2(func(warpModel.bin_centers, *popt)[-1],warpModel.bin_centers[-1]))))

        angles_mean.append(np.mean(all_angles))
        angles_std.append(np.std(all_angles))

    temp = save_name+'GrNr_%d_snap_%d_99_warp_angle_std.npy'%(GrNr,75)
    np.save(temp,angles_std,allow_pickle=True)

 33%|███▎      | 2/6 [01:30<03:00, 45.12s/it]


ValueError: `x` must contain only finite values.

In [14]:
p = .9
N = 100
save_name = '/home/tnguser/postprocessing/uncertainties/neil_angles/'

for subfindID in tqdm(final_diskyIDs):
    GrNr = (il.groupcat.loadSingle(basePath, 99, subhaloID=subfindID)['SubhaloGrNr'])
    principal_file = principal_path + 'GrNr_%d_snap_%d_99_principal_axes_full.npy'%(GrNr,75)
    principal_axes = np.load(principal_file,allow_pickle=True)

    snapArr,zArr = snapshot_redshift_corr(basePath)

    haloTree = il.lhalotree.loadTree(basePath,99,subfindID,
                 fields=['SubhaloGrNr','SnapNum','Group_R_Crit200','SubhaloPos','SubhaloNumber','SubhaloHalfmassRadType'], 
                 onlyMPB=True)

    haloInd,mpb_snapArr = haloTree['SubhaloGrNr'], haloTree['SnapNum']
    subfindID = haloTree['SubhaloNumber']
    Rvirial   = haloTree['Group_R_Crit200']
    subhaloPos = haloTree['SubhaloPos']
    halfmassrad = haloTree['SubhaloHalfmassRadType']

    thetas = []
    phis = []

    for i in tqdm(np.arange(N)):

        spins = []

        for snap in (range(75,100)):

            GrNr_i       = value(haloInd[snap == mpb_snapArr])
            subfindID_i  = value(subfindID[snap == mpb_snapArr])
            Rvirial_i    = value(Rvirial[snap == mpb_snapArr])
            subhaloPos_i = value(subhaloPos[snap == mpb_snapArr])
            a = 1/(1+value(zArr[snap==snapArr]))
            halfmassrad_i = value(halfmassrad[snap == mpb_snapArr])[4]*a

            starPos = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Coordinates')
            starVel = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Velocities')
            starMass = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Masses') 

            # following Neil's method, but only using stars (not gas or DM)
            if (type(starPos) is not dict):
                particleCoords_bar  = starPos
                particleVels_bar    = starVel
                particleMass_bar    = starMass * 1e10
            else:
            # no stars in the halo
                print("ERROR: no stars in halo.")
                particleCoords_bar  = np.array([[np.nan,np.nan,np.nan]])
                particleVels_bar    = np.array([[np.nan,np.nan,np.nan]])
                particleMass_bar    = np.array([np.nan])

            ########################################################################
            # check that the halo is not crossing the box boundary, correct if it is
            # condition is whether it is within 2 virial radii
            passUpperBound = subhaloPos_i + 2*Rvirial_i >= 35000
            passLowerBound = subhaloPos_i - 2*Rvirial_i <= 0

            if any(passUpperBound):
            # shift the particles
                particleCoords_bar[:,passUpperBound] = particleCoords_bar[:,passUpperBound] - 35000/2
                subhaloPos_i[passUpperBound] = subhaloPos_i[passUpperBound] - 35000/2
            #reinforce boundary condition
            ind_bar = np.where(particleCoords_bar[:,passUpperBound]<0)[0]
            particleCoords_bar[ind_bar,passUpperBound] = particleCoords_bar[ind_bar,passUpperBound] + 35000

            if any(passLowerBound):
                particleCoords_bar[:,passLowerBound] = particleCoords_bar[:,passLowerBound] + 35000/2 
                subhaloPos_i[passLowerBound] = subhaloPos_i[passLowerBound] + 35000/2

            ind_bar = np.where(particleCoords_bar[:,passLowerBound]>35000)[0]
            particleCoords_bar[ind_bar,passLowerBound] = particleCoords_bar[ind_bar,passLowerBound] - 35000
            ########################################################################

            # center coords on subhalo pos
            particleCoords_bar = (particleCoords_bar - subhaloPos_i)*a
            # find relative velocities - shouldn't have done this, instead use the spin by TNG (includes DM)
            paricleVels_bar = (particleVels_bar - np.mean(particleVels_bar,axis=0)) * np.sqrt(a) # km/s

            snapshot_spins = []
            innerRad = 0
            outerRad = halfmassrad_i

            r_bar = np.sqrt(np.sum(particleCoords_bar**2,axis=1))
            indsInBounds = np.where((r_bar>innerRad)&(r_bar<=outerRad))[0]        

            # Choosing p percent of samples with repetition
            num_data = len(indsInBounds)
            num_samples = int(p * num_data)
            random_indices = (np.random.randint(0,num_data,size=num_samples))
            bootstrap_inds = np.array(indsInBounds)[random_indices.astype(int)]

            # Choosing p percent of samples with repetition

            J_bar = np.sum(particleMass_bar[bootstrap_inds].reshape(-1,1)*np.cross(particleCoords_bar[bootstrap_inds],
                                                                                particleVels_bar[bootstrap_inds]),axis=0) 
            spins.append(make_unit(J_bar))

        spins = np.array(spins)

        disk_theta = RotateForPolarPlot(spins,principal_axes,close_to_unit_thresh=1e-4,useMajorAxis=False)[0][0]
        disk_phi = RotateForPolarPlot(spins,principal_axes,close_to_unit_thresh=1e-4,useMajorAxis=False)[0][1]

        thetas.append(disk_theta)
        phis.append(disk_phi)

    theta_mean = np.mean(thetas,0)
    theta_std = np.std(thetas,0)

    phi_mean = np.mean(phis,0)
    phi_std = np.std(phis,0)

    np.save(save_name+'GrNr_%d_snap_%d_99_lambda_theta.npy'%(GrNr,75),thetas,allow_pickle=True)    
    np.save(save_name+'GrNr_%d_snap_%d_99_lambda_phi.npy'%(GrNr,75),phis,allow_pickle=True)

    np.save(save_name+'GrNr_%d_snap_%d_99_lambda_theta_mean.npy'%(GrNr,75),theta_mean,allow_pickle=True)    
    np.save(save_name+'GrNr_%d_snap_%d_99_lambda_phi_mean.npy'%(GrNr,75),phi_mean,allow_pickle=True)

    np.save(save_name+'GrNr_%d_snap_%d_99_lambda_theta_std.npy'%(GrNr,75),theta_std,allow_pickle=True)    
    np.save(save_name+'GrNr_%d_snap_%d_99_lambda_phi_std.npy'%(GrNr,75),phi_std,allow_pickle=True)

working on # 1 of  86


100%|██████████| 100/100 [09:33<00:00,  5.74s/it]


working on # 2 of  86


100%|██████████| 100/100 [07:52<00:00,  4.73s/it]


working on # 3 of  86


...

100%|██████████| 100/100 [01:51<00:00,  1.12s/it]


working on # 85 of  86


100%|██████████| 100/100 [01:05<00:00,  1.52it/s]


working on # 86 of  86


100%|██████████| 100/100 [01:11<00:00,  1.41it/s]


In [10]:
p = .9
N = 100
save_name = '/home/tnguser/postprocessing/uncertainties/angles/'

total = len(real_diskyIDs)
num_completed = 1
for subfindID in real_diskyIDs:
    print("working on #", num_completed, "of ", total)

    ### CODE IN COMMENTS ADAPTED FROM NEIL'S "measure_halo_spin.py", 75-99 non-tilted 3kpc ----------------------------------------------
    ### ----------------------------------------------------------------------------------------------------------
    GrNr = (il.groupcat.loadSingle(basePath, 99, subhaloID=subfindID)['SubhaloGrNr'])

    snapArr,zArr = snapshot_redshift_corr(basePath)

    haloTree = il.lhalotree.loadTree(basePath,99,subfindID,
                         fields=['SubhaloGrNr','SnapNum','Group_R_Crit200','SubhaloPos','SubhaloNumber','SubhaloHalfmassRadType'], 
                         onlyMPB=True)

    haloInd,mpb_snapArr = haloTree['SubhaloGrNr'], haloTree['SnapNum']
    subfindID = haloTree['SubhaloNumber']
    Rvirial   = haloTree['Group_R_Crit200']
    subhaloPos = haloTree['SubhaloPos']
    halfmassrad = haloTree['SubhaloHalfmassRadType']
    angles_mean = []
    angles_std = []
    for snap in tqdm(range(75,100)):

        GrNr_i       = value(haloInd[snap == mpb_snapArr])
        subfindID_i  = value(subfindID[snap == mpb_snapArr])
        Rvirial_i    = value(Rvirial[snap == mpb_snapArr])
        subhaloPos_i = value(subhaloPos[snap == mpb_snapArr])
        a = 1/(1+value(zArr[snap==snapArr]))
        halfmassrad_i = value(halfmassrad[snap == mpb_snapArr])[4]*a

        starPos = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Coordinates')
        starVel = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Velocities')
        starMass = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Masses') 

        # following Neil's method, but only using stars (not gas or DM)
        if (type(starPos) is not dict):
            particleCoords_bar  = starPos
            particleVels_bar    = starVel
            particleMass_bar    = starMass * 1e10
        else:
            # no stars in the halo
            print("ERROR: no stars in halo.")
            particleCoords_bar  = np.array([[np.nan,np.nan,np.nan]])
            particleVels_bar    = np.array([[np.nan,np.nan,np.nan]])
            particleMass_bar    = np.array([np.nan])

        ########################################################################
        # check that the halo is not crossing the box boundary, correct if it is
        # condition is whether it is within 2 virial radii
        passUpperBound = subhaloPos_i + 2*Rvirial_i >= 35000
        passLowerBound = subhaloPos_i - 2*Rvirial_i <= 0

        if any(passUpperBound):
            # shift the particles
            particleCoords_bar[:,passUpperBound] = particleCoords_bar[:,passUpperBound] - 35000/2
            subhaloPos_i[passUpperBound] = subhaloPos_i[passUpperBound] - 35000/2
            #reinforce boundary condition
            ind_bar = np.where(particleCoords_bar[:,passUpperBound]<0)[0]
            particleCoords_bar[ind_bar,passUpperBound] = particleCoords_bar[ind_bar,passUpperBound] + 35000

        if any(passLowerBound):
            particleCoords_bar[:,passLowerBound] = particleCoords_bar[:,passLowerBound] + 35000/2 
            subhaloPos_i[passLowerBound] = subhaloPos_i[passLowerBound] + 35000/2

            ind_bar = np.where(particleCoords_bar[:,passLowerBound]>35000)[0]
            particleCoords_bar[ind_bar,passLowerBound] = particleCoords_bar[ind_bar,passLowerBound] - 35000
        ########################################################################

        # center coords on subhalo pos
        particleCoords_bar = (particleCoords_bar - subhaloPos_i)*a
        # find relative velocities - shouldn't have done this, instead use the spin by TNG (includes DM)
        paricleVels_bar = (particleVels_bar - np.mean(particleVels_bar,axis=0)) * np.sqrt(a) # km/s

        ### end Neil's code  -----------------------------------------------------------------------------------------
        ### ----------------------------------------------------------------------------------------------------------
        angles = []
        innerRad = 0
        outerRad = halfmassrad_i
        for i in (np.arange(N)):
            #        print("Working on bootstrap ", i+1, " of ", N)

            r_bar = np.sqrt(np.sum(particleCoords_bar**2,axis=1))
            indsInBounds = np.where((r_bar>innerRad)&(r_bar<=outerRad))[0]        

            # Choosing p percent of samples with repetition
            num_data = len(indsInBounds)
            num_samples = int(p * num_data)
            random_indices = (np.random.randint(0,num_data,size=num_samples))
            bootstrap_inds = np.array(indsInBounds)[random_indices.astype(int)]

            J_bar = np.sum(particleMass_bar[bootstrap_inds].reshape(-1,1)*np.cross(particleCoords_bar[bootstrap_inds],
                                                                                    particleVels_bar[bootstrap_inds]),axis=0) 
            pattern_file = pattern_path + 'GrNr_%d_snap_%d_99_patternSpeeds.npy'%(GrNr,75)
            principal_file = principal_path + 'GrNr_%d_snap_%d_99_principal_axes_full.npy'%(GrNr,75)

            # Must have angular momentum, pattern speeds, and principal axes calculations
            if exists(pattern_file) and exists(principal_file):
                pattern_info = np.load(pattern_file, allow_pickle=True)
                pattern_info = dict(enumerate(pattern_info.flatten(),1))[1]
                principal_axes = np.load(principal_file,allow_pickle=True)
            L_spin = make_unit(J_bar)

            raxis = np.dot(principal_axes[snap-75], getRotationAxis((snap-75), pattern_info['startStop'], pattern_info['Raxis']))
            angle = np.degrees(np.arccos(np.abs(np.dot(raxis, L_spin))))

            if angle >= 45:
                angle = 90 - angle
            angles.append(angle)

        angles_mean.append(np.mean(angles))
        angles_std.append(np.std(angles))

    temp = save_name+'GrNr_%d_snap_%d_99_RaxisLaxis_L_angle_std.npy'%(GrNr,75)
    np.save(temp,angles_std,allow_pickle=True)
    num_completed = (num_completed + 1)

working on # 1 of  23


100%|██████████| 25/25 [00:34<00:00,  1.39s/it]


working on # 2 of  23


100%|██████████| 25/25 [01:27<00:00,  3.49s/it]


working on # 3 of  23


...

100%|██████████| 25/25 [00:15<00:00,  1.57it/s]


working on # 22 of  23


100%|██████████| 25/25 [00:15<00:00,  1.63it/s]


working on # 23 of  23


100%|██████████| 25/25 [00:19<00:00,  1.30it/s]


In [15]:
p = .9
N = 100
#Let's calculate these according to half mass radius instead

save_name = '/home/tnguser/postprocessing/uncertainties/angles/'

total = len(rest_subfindIDs)
num_completed = 1
for subfindID in rest_subfindIDs:
    print("working on #", num_completed, "of ", total)

    ### CODE IN COMMENTS ADAPTED FROM NEIL'S "measure_halo_spin.py", 75-99 non-tilted 3kpc ----------------------------------------------
    ### ----------------------------------------------------------------------------------------------------------
    GrNr = (il.groupcat.loadSingle(basePath, 99, subhaloID=subfindID)['SubhaloGrNr'])

    snapArr,zArr = snapshot_redshift_corr(basePath)

    haloTree = il.lhalotree.loadTree(basePath,99,subfindID,
                         fields=['SubhaloGrNr','SnapNum','Group_R_Crit200','SubhaloPos','SubhaloNumber','SubhaloHalfmassRadType'], 
                         onlyMPB=True)

    haloInd,mpb_snapArr = haloTree['SubhaloGrNr'], haloTree['SnapNum']
    subfindID = haloTree['SubhaloNumber']
    Rvirial   = haloTree['Group_R_Crit200']
    subhaloPos = haloTree['SubhaloPos']
    halfmassrad = haloTree['SubhaloHalfmassRadType']
    angles_mean = []
    angles_std = []
    for snap in tqdm(range(75,100)):

        GrNr_i       = value(haloInd[snap == mpb_snapArr])
        subfindID_i  = value(subfindID[snap == mpb_snapArr])
        Rvirial_i    = value(Rvirial[snap == mpb_snapArr])
        subhaloPos_i = value(subhaloPos[snap == mpb_snapArr])
        a = 1/(1+value(zArr[snap==snapArr]))
        halfmassrad_i = value(halfmassrad[snap == mpb_snapArr])[4]*a

        starPos = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Coordinates')
        starVel = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Velocities')
        starMass = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Masses') 

        # following Neil's method, but only using stars (not gas or DM)
        if (type(starPos) is not dict):
            particleCoords_bar  = starPos
            particleVels_bar    = starVel
            particleMass_bar    = starMass * 1e10
        else:
            # no stars in the halo
            print("ERROR: no stars in halo.")
            particleCoords_bar  = np.array([[np.nan,np.nan,np.nan]])
            particleVels_bar    = np.array([[np.nan,np.nan,np.nan]])
            particleMass_bar    = np.array([np.nan])

        ########################################################################
        # check that the halo is not crossing the box boundary, correct if it is
        # condition is whether it is within 2 virial radii
        passUpperBound = subhaloPos_i + 2*Rvirial_i >= 35000
        passLowerBound = subhaloPos_i - 2*Rvirial_i <= 0

        if any(passUpperBound):
            # shift the particles
            particleCoords_bar[:,passUpperBound] = particleCoords_bar[:,passUpperBound] - 35000/2
            subhaloPos_i[passUpperBound] = subhaloPos_i[passUpperBound] - 35000/2
            #reinforce boundary condition
            ind_bar = np.where(particleCoords_bar[:,passUpperBound]<0)[0]
            particleCoords_bar[ind_bar,passUpperBound] = particleCoords_bar[ind_bar,passUpperBound] + 35000

        if any(passLowerBound):
            particleCoords_bar[:,passLowerBound] = particleCoords_bar[:,passLowerBound] + 35000/2 
            subhaloPos_i[passLowerBound] = subhaloPos_i[passLowerBound] + 35000/2

            ind_bar = np.where(particleCoords_bar[:,passLowerBound]>35000)[0]
            particleCoords_bar[ind_bar,passLowerBound] = particleCoords_bar[ind_bar,passLowerBound] - 35000
        ########################################################################

        # center coords on subhalo pos
        particleCoords_bar = (particleCoords_bar - subhaloPos_i)*a
        # find relative velocities - shouldn't have done this, instead use the spin by TNG (includes DM)
        paricleVels_bar = (particleVels_bar - np.mean(particleVels_bar,axis=0)) * np.sqrt(a) # km/s

        ### end Neil's code  -----------------------------------------------------------------------------------------
        ### ----------------------------------------------------------------------------------------------------------
        angles = []
        innerRad = 0
        outerRad = halfmassrad_i
        for i in (np.arange(N)):
            #        print("Working on bootstrap ", i+1, " of ", N)

            r_bar = np.sqrt(np.sum(particleCoords_bar**2,axis=1))
            indsInBounds = np.where((r_bar>innerRad)&(r_bar<=outerRad))[0]        

            # Choosing p percent of samples with repetition
            num_data = len(indsInBounds)
            num_samples = int(p * num_data)
            random_indices = (np.random.randint(0,num_data,size=num_samples))
            bootstrap_inds = np.array(indsInBounds)[random_indices.astype(int)]

            J_bar = np.sum(particleMass_bar[bootstrap_inds].reshape(-1,1)*np.cross(particleCoords_bar[bootstrap_inds],
                                                                                    particleVels_bar[bootstrap_inds]),axis=0) 
            pattern_file = pattern_path + 'GrNr_%d_snap_%d_99_patternSpeeds.npy'%(GrNr,75)
            principal_file = principal_path + 'GrNr_%d_snap_%d_99_principal_axes_full.npy'%(GrNr,75)

            # Must have angular momentum, pattern speeds, and principal axes calculations
            if exists(pattern_file) and exists(principal_file):
                pattern_info = np.load(pattern_file, allow_pickle=True)
                pattern_info = dict(enumerate(pattern_info.flatten(),1))[1]
                principal_axes = np.load(principal_file,allow_pickle=True)
            L_spin = make_unit(J_bar)

            angle = np.degrees(np.arccos(np.abs(np.dot(principal_axes[snap-75][:,1], L_spin))))
            angles.append(angle)

        angles_mean.append(np.mean(angles))
        angles_std.append(np.std(angles))

    temp = save_name+'GrNr_%d_snap_%d_99_RaxisLaxis_x_angle_std.npy'%(GrNr,75)
    np.save(temp,angles_std,allow_pickle=True)
    num_completed = (num_completed + 1)

working on # 1 of  86


100%|██████████| 25/25 [01:14<00:00,  2.99s/it]


working on # 2 of  86


100%|██████████| 25/25 [01:08<00:00,  2.73s/it]


working on # 3 of  86


...

100%|██████████| 25/25 [00:08<00:00,  2.86it/s]


working on # 85 of  86


100%|██████████| 25/25 [00:06<00:00,  4.08it/s]


working on # 86 of  86


100%|██████████| 25/25 [00:06<00:00,  3.94it/s]


In [9]:
p = .9
N = 100
#Let's calculate these according to half mass radius instead

save_name = '/home/tnguser/postprocessing/uncertainties/angles/'

for subfindID in tqdm(final_diskyIDs):

    ### CODE IN COMMENTS ADAPTED FROM NEIL'S "measure_halo_spin.py", 75-99 non-tilted 3kpc ----------------------------------------------
    ### ----------------------------------------------------------------------------------------------------------
    GrNr = (il.groupcat.loadSingle(basePath, 99, subhaloID=subfindID)['SubhaloGrNr'])

    snapArr,zArr = snapshot_redshift_corr(basePath)

    haloTree = il.lhalotree.loadTree(basePath,99,subfindID,
                         fields=['SubhaloGrNr','SnapNum','Group_R_Crit200','SubhaloPos','SubhaloNumber','SubhaloHalfmassRadType'], 
                         onlyMPB=True)

    haloInd,mpb_snapArr = haloTree['SubhaloGrNr'], haloTree['SnapNum']
    subfindID = haloTree['SubhaloNumber']
    Rvirial   = haloTree['Group_R_Crit200']
    subhaloPos = haloTree['SubhaloPos']
    halfmassrad = haloTree['SubhaloHalfmassRadType']
    angles_mean = []
    angles_std = []
    for snap in (range(75,100)):

        GrNr_i       = value(haloInd[snap == mpb_snapArr])
        subfindID_i  = value(subfindID[snap == mpb_snapArr])
        Rvirial_i    = value(Rvirial[snap == mpb_snapArr])
        subhaloPos_i = value(subhaloPos[snap == mpb_snapArr])
        a = 1/(1+value(zArr[snap==snapArr]))
        halfmassrad_i = value(halfmassrad[snap == mpb_snapArr])[4]*a

        starPos = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Coordinates')
        starVel = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Velocities')
        starMass = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Masses') 

        # following Neil's method, but only using stars (not gas or DM)
        if (type(starPos) is not dict):
            particleCoords_bar  = starPos
            particleVels_bar    = starVel
            particleMass_bar    = starMass * 1e10
        else:
            # no stars in the halo
            print("ERROR: no stars in halo.")
            particleCoords_bar  = np.array([[np.nan,np.nan,np.nan]])
            particleVels_bar    = np.array([[np.nan,np.nan,np.nan]])
            particleMass_bar    = np.array([np.nan])

        ########################################################################
        # check that the halo is not crossing the box boundary, correct if it is
        # condition is whether it is within 2 virial radii
        passUpperBound = subhaloPos_i + 2*Rvirial_i >= 35000
        passLowerBound = subhaloPos_i - 2*Rvirial_i <= 0

        if any(passUpperBound):
            # shift the particles
            particleCoords_bar[:,passUpperBound] = particleCoords_bar[:,passUpperBound] - 35000/2
            subhaloPos_i[passUpperBound] = subhaloPos_i[passUpperBound] - 35000/2
            #reinforce boundary condition
            ind_bar = np.where(particleCoords_bar[:,passUpperBound]<0)[0]
            particleCoords_bar[ind_bar,passUpperBound] = particleCoords_bar[ind_bar,passUpperBound] + 35000

        if any(passLowerBound):
            particleCoords_bar[:,passLowerBound] = particleCoords_bar[:,passLowerBound] + 35000/2 
            subhaloPos_i[passLowerBound] = subhaloPos_i[passLowerBound] + 35000/2

            ind_bar = np.where(particleCoords_bar[:,passLowerBound]>35000)[0]
            particleCoords_bar[ind_bar,passLowerBound] = particleCoords_bar[ind_bar,passLowerBound] - 35000
        ########################################################################

        # center coords on subhalo pos
        particleCoords_bar = (particleCoords_bar - subhaloPos_i)*a
        # find relative velocities - shouldn't have done this, instead use the spin by TNG (includes DM)
        paricleVels_bar = (particleVels_bar - np.mean(particleVels_bar,axis=0)) * np.sqrt(a) # km/s

        ### end Neil's code  -----------------------------------------------------------------------------------------
        ### ----------------------------------------------------------------------------------------------------------
        angles = []
        innerRad = 0
        outerRad = halfmassrad_i
        for i in (np.arange(N)):
            #        print("Working on bootstrap ", i+1, " of ", N)

            r_bar = np.sqrt(np.sum(particleCoords_bar**2,axis=1))
            indsInBounds = np.where((r_bar>innerRad)&(r_bar<=outerRad))[0]        

            # Choosing p percent of samples with repetition
            num_data = len(indsInBounds)
            num_samples = int(p * num_data)
            random_indices = (np.random.randint(0,num_data,size=num_samples))
            bootstrap_inds = np.array(indsInBounds)[random_indices.astype(int)]

            J_bar = np.sum(particleMass_bar[bootstrap_inds].reshape(-1,1)*np.cross(particleCoords_bar[bootstrap_inds],
                                                                                    particleVels_bar[bootstrap_inds]),axis=0) 
            pattern_file = pattern_path + 'GrNr_%d_snap_%d_99_patternSpeeds.npy'%(GrNr,75)
            principal_file = principal_path + 'GrNr_%d_snap_%d_99_principal_axes_full.npy'%(GrNr,75)

            # Must have angular momentum, pattern speeds, and principal axes calculations
            if exists(pattern_file) and exists(principal_file):
                pattern_info = np.load(pattern_file, allow_pickle=True)
                pattern_info = dict(enumerate(pattern_info.flatten(),1))[1]
                principal_axes = np.load(principal_file,allow_pickle=True)
            L_spin = make_unit(J_bar)

            angle = np.degrees(np.arccos(np.abs(np.dot(principal_axes[snap-75][:,2], L_spin))))
            if angle > 45:
                angle = 90-angle
            angles.append(angle)

        angles_mean.append(np.mean(angles))
        angles_std.append(np.std(angles))

    temp = save_name+'GrNr_%d_snap_%d_99_torque_angle_std.npy'%(GrNr,75)
    np.save(temp,angles_std,allow_pickle=True)
    num_completed = (num_completed + 1)

  0%|          | 0/86 [00:00<?, ?it/s]

working on # 1 of  86


  1%|          | 1/86 [01:16<1:49:03, 76.99s/it]

working on # 2 of  86


  2%|▏         | 2/86 [02:22<1:38:27, 70.32s/it]

...

 98%|█████████▊| 84/86 [43:52<00:30, 15.17s/it]

working on # 85 of  86


 99%|█████████▉| 85/86 [44:06<00:14, 14.83s/it]

working on # 86 of  86


100%|██████████| 86/86 [44:20<00:00, 30.94s/it]
